In [1]:
import torch
names_file = open("./data/names.txt", "r")
G = torch.Generator().manual_seed(42)
names_all = names_file.read().split("\n")
names_train, names_dev, names_test = torch.utils.data.random_split(names_all, [0.8, 0.1, 0.1], generator=G)

import matplotlib.pyplot as plt

architecture of network will be 27 neurons with 27 weights each thus they will simulate table from bigram (27x27)
the input will one_hot vector 
so one hot vector will be applied to every neuron but only one weight from neuron will be used rest will be zero
then we have to move this numbers to something that looks life probability <- softmax
then maximum likelihood as cost function
and finally gradient

In [2]:
FIRST_CHAR = '<S>'
LAST_CHAR = '<E>'

all_letters = set()
for name in names_all:
    normalized_name = list(name)
    for char in normalized_name:
        all_letters.add(char)

letter_to_int32 = {letter: index for index, letter in enumerate(all_letters)}
letter_to_int32[FIRST_CHAR] = 26
letter_to_int32[LAST_CHAR] = 27

In [3]:
xs = []
ys = []
for name in names_train:
    normalized_name = [FIRST_CHAR] + list(name) + [LAST_CHAR]
    for ch1, ch2 in zip(normalized_name, normalized_name[1:]):
        xs.append(letter_to_int32[ch1])
        ys.append(letter_to_int32[ch2])
xs = torch.tensor(xs)
ys = torch.tensor(ys)

xs_enc = torch.nn.functional.one_hot(xs, num_classes=28).float()
ys_enc = torch.nn.functional.one_hot(ys, num_classes=28).float()
# weights initialization


In [4]:
W = torch.randn((28,28), requires_grad=True, generator=G)

for _ in range(100):
    # in one step we evaluate whole model in parallel
    evaluated  = W[xs]

    # forward 

    # maximum likehood
    # evaluated_exp = evaluated.exp()
    # prob = evaluated_exp / evaluated_exp.sum(1, keepdims=True)
    #loss = prob[xs, ys].log().mean() * -1 
  
    # cross entropy
    loss = torch.nn.functional.cross_entropy(evaluated, ys)

    loss.backward()

    W.data += -20 * W.grad
    W.grad = None
    
    print(f'{loss=}')



loss=tensor(3.7961, grad_fn=<NllLossBackward0>)
loss=tensor(3.6389, grad_fn=<NllLossBackward0>)
loss=tensor(3.5103, grad_fn=<NllLossBackward0>)
loss=tensor(3.4039, grad_fn=<NllLossBackward0>)
loss=tensor(3.3147, grad_fn=<NllLossBackward0>)
loss=tensor(3.2388, grad_fn=<NllLossBackward0>)
loss=tensor(3.1734, grad_fn=<NllLossBackward0>)
loss=tensor(3.1164, grad_fn=<NllLossBackward0>)
loss=tensor(3.0663, grad_fn=<NllLossBackward0>)
loss=tensor(3.0220, grad_fn=<NllLossBackward0>)
loss=tensor(2.9825, grad_fn=<NllLossBackward0>)
loss=tensor(2.9474, grad_fn=<NllLossBackward0>)
loss=tensor(2.9159, grad_fn=<NllLossBackward0>)
loss=tensor(2.8878, grad_fn=<NllLossBackward0>)
loss=tensor(2.8625, grad_fn=<NllLossBackward0>)
loss=tensor(2.8399, grad_fn=<NllLossBackward0>)
loss=tensor(2.8194, grad_fn=<NllLossBackward0>)
loss=tensor(2.8009, grad_fn=<NllLossBackward0>)
loss=tensor(2.7841, grad_fn=<NllLossBackward0>)
loss=tensor(2.7688, grad_fn=<NllLossBackward0>)
loss=tensor(2.7548, grad_fn=<NllLossBack

In [5]:
# maximum likehood
W = torch.randn((28,28), requires_grad=True, generator=G)

for _ in range(100):
    # in one step we evaluate whole model in parallel
    evaluated  = W[xs]

    # forward 
    evaluated_exp = evaluated.exp()
    prob = evaluated_exp / evaluated_exp.sum(1, keepdims=True)
    loss = prob[xs, ys].log().mean() * -1 
  

    loss.backward()

    W.data += -20 * W.grad
    W.grad = None
    
    print(f'{loss=}')



loss=tensor(3.7773, grad_fn=<MulBackward0>)
loss=tensor(3.5775, grad_fn=<MulBackward0>)
loss=tensor(3.4165, grad_fn=<MulBackward0>)
loss=tensor(3.2855, grad_fn=<MulBackward0>)
loss=tensor(3.1811, grad_fn=<MulBackward0>)
loss=tensor(3.0996, grad_fn=<MulBackward0>)
loss=tensor(3.0358, grad_fn=<MulBackward0>)
loss=tensor(2.9848, grad_fn=<MulBackward0>)
loss=tensor(2.9430, grad_fn=<MulBackward0>)
loss=tensor(2.9081, grad_fn=<MulBackward0>)
loss=tensor(2.8788, grad_fn=<MulBackward0>)
loss=tensor(2.8540, grad_fn=<MulBackward0>)
loss=tensor(2.8329, grad_fn=<MulBackward0>)
loss=tensor(2.8149, grad_fn=<MulBackward0>)
loss=tensor(2.7993, grad_fn=<MulBackward0>)
loss=tensor(2.7856, grad_fn=<MulBackward0>)
loss=tensor(2.7735, grad_fn=<MulBackward0>)
loss=tensor(2.7627, grad_fn=<MulBackward0>)
loss=tensor(2.7529, grad_fn=<MulBackward0>)
loss=tensor(2.7440, grad_fn=<MulBackward0>)
loss=tensor(2.7359, grad_fn=<MulBackward0>)
loss=tensor(2.7285, grad_fn=<MulBackward0>)
loss=tensor(2.7216, grad_fn=<Mul

In [9]:
# with 2 layers
W1 = torch.randn((28,28), requires_grad=True, generator=G)
W2 = torch.randn((28,28), requires_grad=True, generator=G)

for _ in range(50):
    # in one step we evaluate whole model in parallel
    evaluated  = xs_enc @ W1 @  W2

    # forward 
    loss = torch.nn.functional.cross_entropy(evaluated, ys) 

    loss.backward()

    W1.data += -5 * W1.grad
    W1.grad = None

    W2.data += -5 * W2.grad
    W2.grad = None
    
    print(f'{loss=}')

model = xs_enc  @ W1 @ W2
model_exp = evaluated.exp()
P = model_exp / model_exp.sum(1, keepdims=True)



loss=tensor(10.6648, grad_fn=<NllLossBackward0>)
loss=tensor(7.2466, grad_fn=<NllLossBackward0>)
loss=tensor(6.6062, grad_fn=<NllLossBackward0>)
loss=tensor(6.8986, grad_fn=<NllLossBackward0>)
loss=tensor(6.7238, grad_fn=<NllLossBackward0>)
loss=tensor(4.8207, grad_fn=<NllLossBackward0>)
loss=tensor(4.0288, grad_fn=<NllLossBackward0>)
loss=tensor(3.4917, grad_fn=<NllLossBackward0>)
loss=tensor(3.2539, grad_fn=<NllLossBackward0>)
loss=tensor(3.0679, grad_fn=<NllLossBackward0>)
loss=tensor(2.9507, grad_fn=<NllLossBackward0>)
loss=tensor(2.8928, grad_fn=<NllLossBackward0>)
loss=tensor(2.8207, grad_fn=<NllLossBackward0>)
loss=tensor(2.7956, grad_fn=<NllLossBackward0>)
loss=tensor(2.7463, grad_fn=<NllLossBackward0>)
loss=tensor(2.7269, grad_fn=<NllLossBackward0>)
loss=tensor(2.6952, grad_fn=<NllLossBackward0>)
loss=tensor(2.6858, grad_fn=<NllLossBackward0>)
loss=tensor(2.6593, grad_fn=<NllLossBackward0>)
loss=tensor(2.6549, grad_fn=<NllLossBackward0>)
loss=tensor(2.6346, grad_fn=<NllLossBac

In [28]:

int32_to_letter = {value: key for key,value in letter_to_int32.items()}
def predict_next_letter_index(letter_index):
    prob_dist = P[letter_index]
    return torch.multinomial(prob_dist, num_samples=1).item()



def create_word():
    idx = 26
    word = []
    while True:
        idx = predict_next_letter_index(idx)
        word.append(int32_to_letter[idx])
        if idx == 27 and len(word) > 1:
            return ''.join(word[:-1])
        
for _ in range(10):
    print(create_word())

print(P[26])

mieewxzil
<E>llmhytngeaa
vwj
<E>moxhdzea
h
lye
bshycd
<E>
eii
wtaykoknoao<S>r
tensor([0.0063, 0.0834, 0.0030, 0.0198, 0.0980, 0.0042, 0.0033, 0.0354, 0.0206,
        0.0183, 0.0024, 0.0180, 0.0109, 0.0056, 0.0320, 0.0251, 0.0035, 0.1051,
        0.0684, 0.0644, 0.0098, 0.0228, 0.0135, 0.0383, 0.0033, 0.0588, 0.0016,
        0.2243], grad_fn=<SelectBackward0>)
